<a href="https://colab.research.google.com/github/Hasnainghori112/Tool-calling-ai-agent/blob/main/streaming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uq openai-agents


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.5/119.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.1 MB/s eta 0:00:00


In [2]:
import nest_asyncio
nest_asyncio.apply()

In [4]:
import os
from agents import Agent, Runner,  ItemHelpers,AsyncOpenAI, OpenAIChatCompletionsModel
from agents.run import RunConfig
from google.colab import userdata
import requests
from agents import function_tool

from agents import (
    Agent,
    Runner,
    set_default_openai_api,
    set_default_openai_client,
    set_tracing_disabled,
)

gemini_api_key = userdata.get("GOOGLE_API_KEY")


# Check if the API key is present; if not, raise an error
if not gemini_api_key:
    raise ValueError("GEMINI_API_KEY is not set. Please ensure it is defined in your .env file.")

#Reference: https://ai.google.dev/gemini-api/docs/openai
external_client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client
)

set_default_openai_client(client=external_client, use_for_tracing=False)
set_default_openai_api("chat_completions")
set_tracing_disabled(disabled=True)

In [5]:
@function_tool
def getWeather(city: str) -> str:
    """
    Get the weather for a given city.
    """
    # Replace with your actual weather API URL and key
    result = requests.get(f"http://api.weatherapi.com/v1/current.json?key=8e3aca2b91dc4342a1162608252604&q={city}")

    if result.status_code == 200:
        data = result.json()
        return f"The weather in {city} is {data['current']['temp_c']}°C with {data['current']['condition']['text']}."
    else:
        return "Sorry, I couldn't fetch the weather data."



In [9]:
import asyncio
Assistant = Agent(
    name = "assistant",
    instructions= """You are a helpful and knowledgeable assistant.
                      Your role is to answer all user questions accurately and clearly, whether they relate to general knowledge, current events, or specific tasks.
                      You may use available tools when necessary to enhance your responses, ensuring the user receives the most relevant and up-to-date information.
                      Always aim to be concise, informative, and easy to understand.""",
    tools = [getWeather],
    model=model
)

async def main():
  result =  Runner.run_streamed(Assistant,"write in 100 lines summary of pakistan and  get the current weather of his capital")
  async for event in result.stream_events():
    if event.type == "raw_response_event":
      continue
      print(event)
    elif event.type == "agent_updated_stream_event":
      print(f"Agent updated: {event.new_agent.name}")
      continue
    elif event.type == "run_item_stream_event":
      if event.item.type == "tool_call_item":
        print("-- Tool was called")
      elif event.item.type == "tool_call_output_item":
        print(f"-- Tool output: {event.item.output}")
      elif event.item.type == "message_output_item":
         print(f"-- Message output:\n {ItemHelpers.text_message_output(event.item)}" )
      else:
        pass
asyncio.run(main())

Agent updated: assistant
-- Message output:
 Pakistan, officially the Islamic Republic of Pakistan, is a country in South Asia. It is the fifth-most populous country in the world, with a population exceeding 240 million. Pakistan has a diverse landscape, including deserts, forests, mountains, and coastlines. The country's economy is the 24th-largest in the world in terms of purchasing power parity, and is classified as a developing nation. Pakistan has a complex history, gaining independence from British India in 1947. It has a parliamentary republic form of government.

-- Tool was called
-- Tool output: The weather in Islamabad is 32.4°C with Sunny.
-- Message output:
 The current weather in Islamabad is 32.4°C with Sunny conditions.

